In [157]:
import torch as tr 
import numpy as np
from torch.autograd import Variable
from torch import optim
from sklearn.utils.extmath import fast_logdet
from sklearn.datasets import make_sparse_spd_matrix

In [2]:
random_state=0

In [49]:
A = make_sparse_spd_matrix(dim=15, alpha=0.7, random_state=random_state)

In [65]:
T_true = A[5:,5:]
K_true = A[10:,5:,]
H_true = A[0:5,0:5]
T_true.shape, K_true.shape, H_true.shape

((10, 10), (5, 10), (5, 5))

In [66]:
per_cov = K_true*0.05
T_obs = T_true - per_cov.T.dot(H_true).dot(per_cov)

In [67]:
samples = np.random.multivariate_normal(np.zeros(10), np.linalg.inv(T_obs), 50)

In [68]:
S = Variable(tr.from_numpy(samples.T.dot(samples)/50))


In [88]:
_lambda=100
_mu=10

In [133]:
# def loss_func(S, T, K, H, _lambda, _mu):
#     loss = 0
#     Hinv = H.inverse().double()
#     Hinv.type(tr.DoubleTensor)
#     T = T.double()
#     H = H.double()
#     R = T - tr.matmul(tr.matmul(tr.t(K).double(), Hinv),K.double())
#     loss += - R.logdet() + tr.trace(tr.matmul(S,R))
#     loss += _lambda*tr.norm(T, 1)
#     loss += _mu*tr.norm(H, 1)
#     return loss

def nuclear_norm(A):
    _, S_, V = tr.svd(A)
    return tr.sum(S)

def loss_func(S, T, L, _lambda, _mu):
    loss = 0
    T = T.double()
    L = L.double()
    LLT = tr.matmul(L, tr.t(L))
    R = T - LLT
    loss += - R.logdet() + tr.trace(tr.matmul(S,R))
    loss += _lambda*tr.norm(T, 1)
    loss += _mu*nuclear_norm(LLT)
    return loss

In [130]:
tr.distributions.constraints.positive_definite??

In [176]:
K = Variable(tr.from_numpy(per_cov), )
H = Variable(tr.randn(5,5), requires_grad=True)
T = Variable(tr.randn(10,10), requires_grad=True)
# for epoch in range(5000):

#     # Calcola funzione costo
#     loss = loss_func(S, T, K, H, _lambda, _mu)

#     # Esegui back-propagation
#     loss.backward()

#     # Aggiorna le variabili
#     H.data = H.data - 0.0005*H.grad.data
#     T.data = T.data - 0.0005*T.grad.data
#     # Resetta il gradiente
#     H.grad.data.zero_()
#     T.grad.data.zero_()


L = Variable(tr.from_numpy(make_sparse_spd_matrix(dim=10, alpha=0.5)))
L = Variable(tr.potrf(L, upper=False), requires_grad=True)
optimizer = optim.SGD([K, L], lr = 0.01, momentum=0.1)

for epoch in range(1000):
    optimizer.zero_grad()
    #output = model(input)
    loss = loss_func(S, T, L, _lambda, _mu)
    loss.backward()
    optimizer.step()


In [177]:
Hn = H.detach().numpy()
Hn[np.where(np.abs(Hn)<1e-1)] =0
Hn

array([[ 1.6240873 ,  0.        ,  0.5903708 ,  0.13373913,  1.2193832 ],
       [ 1.0871108 ,  0.5186936 ,  0.33555445,  0.89693993, -0.40317497],
       [ 0.38041613,  0.21467547,  1.0133914 ,  0.3741619 , -0.25600353],
       [-0.10021899,  0.74725634, -0.69336694, -1.636791  , -0.36241144],
       [-0.6807566 ,  0.        ,  0.39597255, -1.7683268 , -1.3137127 ]],
      dtype=float32)

In [183]:
T

tensor([[ 1.0192, -0.7289,  0.3443,  0.5341, -2.1659, -0.1782, -0.6823, -3.1568,
          0.3384, -0.8781],
        [-0.6219,  0.3657, -0.6669, -0.2787,  0.3175, -1.9974,  0.1566,  2.0622,
          0.0768, -1.7206],
        [ 0.1202, -0.3254,  0.1035, -0.3413,  0.5194,  0.0782, -0.6197, -0.4847,
         -0.2961,  0.7022],
        [-1.2544,  0.3659, -0.5154, -0.0814,  0.1421,  0.5306,  1.2642,  2.0209,
         -1.4307,  1.1689],
        [ 0.2032, -0.1543,  0.8386,  0.8626, -0.9450,  1.6023,  0.3435, -0.1113,
         -0.1165,  0.6055],
        [ 0.0140, -0.8883, -0.0892, -0.1527,  0.2615, -0.3761,  1.1188,  1.6171,
         -0.1341, -0.0950],
        [ 0.4135,  0.2405,  0.4341,  1.3034, -0.5228, -0.1207,  0.4392, -0.1647,
         -1.1947,  1.8330],
        [ 2.1331,  1.0956, -0.6838, -1.3670, -0.7854, -0.6755, -1.0032,  0.4987,
         -1.5549,  0.0940],
        [-0.9762,  1.1049, -0.1702, -0.2647,  0.2828,  0.8422,  1.4925, -1.0189,
         -0.8575,  0.1200],
        [ 0.7654, -

In [184]:
T_true

array([[ 1.        ,  0.        , -0.16673795,  0.        ,  0.        ,
        -0.88199611,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  2.04044801,  0.        , -0.63593324,  0.        ,
         0.        , -0.32538408,  0.        ,  0.27733968, -0.72812233],
       [-0.16673795,  0.        ,  1.32129891,  0.        ,  0.        ,
         0.14706222,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.63593324,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.88199611,  0.        ,  0.14706222,  0.        ,  0.        ,
         2.55713568,  0.44253476, -0.8012042 ,  0.        ,  0.        ],
       [ 0.        , -0.32538408,  0.        ,  0.        ,  0.        ,
         0.44253476,  1.30507622, -0.55233705

In [185]:
Ltrue

array([[ 2.37597182e-03,  0.00000000e+00, -3.96164667e-04,
         0.00000000e+00,  0.00000000e+00, -6.88855909e-03,
        -1.19212558e-03,  2.15832992e-03,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  5.19474519e-03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.64582991e-03,
        -6.39191241e-03,  3.30231659e-03,  5.80784117e-03,
        -5.09312291e-03],
       [-3.96164667e-04,  0.00000000e+00,  6.60556839e-05,
         0.00000000e+00,  0.00000000e+00,  1.14858421e-03,
         1.98772573e-04, -3.59875503e-04,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.

In [186]:
Ltrue - tr.matmul(L, tr.t(L)).detach().numpy()

array([[-6.29144052e+90,  1.81053712e+92,  3.58130095e+90,
         1.74254688e+92, -6.32252621e+91, -2.11328574e+90,
         2.28075715e+92,  1.78533792e+92,  7.20316049e+91,
         1.45554103e+92],
       [ 1.81053712e+92, -5.21032449e+93, -1.03061903e+92,
        -5.01466366e+93,  1.81948289e+93,  6.08156790e+91,
        -6.56351350e+93, -5.13780678e+93, -2.07290992e+93,
        -4.18872445e+93],
       [ 3.58130095e+90, -1.03061903e+92, -2.03859775e+90,
        -9.91916681e+91,  3.59899597e+91,  1.20295379e+90,
        -1.29828419e+92, -1.01627479e+92, -4.10028282e+91,
        -8.28543235e+91],
       [ 1.74254688e+92, -5.01466366e+93, -9.91916681e+91,
        -4.82635039e+93,  1.75115672e+93,  5.85318968e+91,
        -6.31703700e+93, -4.94486917e+93, -1.99506692e+93,
        -4.03142727e+93],
       [-6.32252621e+91,  1.81948289e+93,  3.59899597e+91,
         1.75115672e+93, -6.35376549e+92, -2.12372738e+91,
         2.29202625e+93,  1.79415918e+93,  7.23875094e+92,
         1.

In [187]:
from itertools import product
summ = np.zeros((3,3))

for i,j in product(range(3), range(3)):
    I = np.zeros((3,3))
    I[i,j] = 1
    summ[i,j] = Kn.dot(Rn - Tn + Kn.T.dot(Hninv).dot(Kn)).dot(Kn.T).dot((-Hninv).dot(I).dot(Hninv))

NameError: name 'Kn' is not defined

In [ ]:
summ

In [ ]:
Kn = K.numpy()
Hn = H.detach().numpy()
Rn = R.numpy()
Tn = T.numpy()

In [ ]:
np.linalg.norm(Rn - Tn + Kn.T.dot(np.linalg.inv(Hn)).dot(Kn))**2

In [ ]:
Hninv = np.linalg.inv(Hn)
I = np.zeros((3,3))
I[1,1] = 1
Kn.dot(Rn - Tn + Kn.T.dot(Hninv).dot(Kn)).dot(Kn.T).dot((-Hninv.T).dot(I).dot(Hninv))

In [ ]:
Hninv = np.linalg.inv(Hn)
(-Hninv).dot(np.ones((3,3))).dot(Hninv).dot(Kn.dot(Rn - Tn + Kn.T.dot(Hninv).dot(Kn)).dot(Kn.T))

In [ ]:
Hninv

In [200]:
A = Variable(tr.from_numpy(np.random.rand(5,5)))
K = Variable(tr.from_numpy(np.random.rand(10,5)))
B = Variable(tr.from_numpy(make_sparse_spd_matrix(10, alpha=0.3)), requires_grad=True)

In [201]:
loss = tr.norm(A - tr.matmul(tr.matmul(tr.t(K), B.inverse()),K))
loss.backward()
print(B.grad.data)

tensor([[   -485.4929,   -1204.8520,    -778.2334,   -2689.1866,    -926.9998,
           -3105.7977,    -549.1175,   -7924.8030,   -1133.7194,   -3428.2247],
        [  -1204.8588,   -2990.1101,   -1931.3601,   -6673.8166,   -2300.5573,
           -7707.7310,   -1362.7585,  -19667.1693,   -2813.5777,   -8507.9054],
        [   -778.2310,   -1931.3434,   -1247.4861,   -4310.6883,   -1485.9542,
           -4978.5052,    -880.2206,  -12703.2320,   -1817.3192,   -5495.3454],
        [  -2689.1922,   -6673.7930,   -4310.7106,  -14895.6654,   -5134.7422,
          -17203.3158,   -3041.6161,  -43896.2550,   -6279.7816,  -18989.2700],
        [   -927.0016,   -2300.5488,   -1485.9616,   -5134.7414,   -1770.0166,
           -5930.2207,   -1048.4874,  -15131.6454,   -2164.7273,   -6545.8635],
        [  -3105.7957,   -7707.6827,   -4978.5172,  -17203.2688,   -5930.2054,
          -19868.4175,   -3512.8184,  -50696.5682,   -7252.6310,  -21931.0458],
        [   -549.1164,   -1362.7482,    -880.2

In [207]:
An = A.numpy()
Kn = K.numpy()
Bn = B.detach().numpy()

In [203]:
par = An - np.linalg.multi_dot((Kn.T, np.linalg.inv(Bn), Kn))
BB = np.linalg.inv(Bn).dot(np.linalg.inv(Bn))
- Kn.dot(par).dot(Kn.T).dot(BB)

array([[6.29671404e+06, 1.57595191e+07, 1.03771952e+07, 3.57161552e+07,
        1.22543608e+07, 4.11145125e+07, 7.09959077e+06, 1.05241375e+08,
        1.51538745e+07, 4.53539937e+07],
       [4.57563504e+06, 1.14519755e+07, 7.54079964e+06, 2.59538693e+07,
        8.90488043e+06, 2.98766960e+07, 5.15906254e+06, 7.64757833e+07,
        1.10118706e+07, 3.29574010e+07],
       [2.81486134e+06, 7.04508146e+06, 4.63898590e+06, 1.59664242e+07,
        5.47814678e+06, 1.83796909e+07, 3.17377882e+06, 4.70467385e+07,
        6.77433375e+06, 2.02748901e+07],
       [5.43614582e+06, 1.36056746e+07, 8.95894997e+06, 3.08348490e+07,
        1.05795643e+07, 3.54954159e+07, 6.12929366e+06, 9.08580966e+07,
        1.30828032e+07, 3.91554895e+07],
       [4.06188673e+06, 1.01661572e+07, 6.69412534e+06, 2.30397898e+07,
        7.90504769e+06, 2.65221665e+07, 4.57980829e+06, 6.78891458e+07,
        9.77546607e+06, 2.92569704e+07],
       [6.22723760e+06, 1.55856320e+07, 1.02626959e+07, 3.53220690e+07,
   

In [219]:
loss = tr.norm((A - tr.matmul(tr.t(K), tr.matmul(B ,K))))**2
loss.backward()
print(tr.matmul((K), tr.matmul((A - tr.matmul(tr.t(K), tr.matmul(B ,K))), tr.t(K))))
print(B.grad.data)

tensor([[-49.9944, -37.2964, -26.8515, -41.2867, -37.3637, -52.9227, -40.6074,
         -53.7665, -29.2351, -64.7759],
        [-37.7227, -28.9131, -21.4431, -30.7809, -29.3259, -40.5946, -31.4546,
         -41.1206, -22.2114, -49.2336],
        [-26.2849, -20.9902, -17.5555, -21.0892, -22.4343, -30.2312, -24.3956,
         -29.8138, -15.6739, -34.9051],
        [-41.3257, -30.4623, -21.6197, -34.3436, -30.2926, -43.4428, -33.2672,
         -44.2324, -24.0795, -53.4162],
        [-36.9511, -28.7874, -22.5674, -29.9080, -29.9548, -40.9123, -32.1892,
         -40.9051, -21.8853, -48.5196],
        [-52.3189, -39.7614, -30.4752, -42.9073, -40.8821, -57.1511, -44.7857,
         -57.3510, -30.7633, -68.3296],
        [-40.3463, -31.0977, -24.6569, -32.9882, -32.3441, -44.9622, -35.8298,
         -44.8689, -23.7899, -53.0762],
        [-53.9849, -41.0019, -30.5213, -44.3168, -41.5765, -58.2150, -45.3060,
         -58.8359, -31.6993, -70.4311],
        [-29.1911, -21.9266, -15.9642, -24.0220,

In [217]:
par = An - np.linalg.multi_dot((Kn.T, Bn, Kn))
print(par)
Kn.dot(par).dot(Kn.T)

[[-10.99717907  -5.63337029  -5.48519692  -4.92908283  -4.26182627]
 [ -5.63437977  -5.68784248  -5.33193289  -3.5079639   -4.79554441]
 [ -5.65520406  -4.79817102  -6.05426364  -3.67779071  -3.88640569]
 [ -5.20607879  -4.16810127  -4.1090996   -3.5807355   -3.44500492]
 [ -4.84562924  -4.98783751  -3.92973111  -3.41654025  -3.75503248]]


array([[-49.9944049 , -37.2964084 , -26.85147366, -41.28670551,
        -37.36371383, -52.92265898, -40.6073847 , -53.76651921,
        -29.23507156, -64.77592673],
       [-37.72265651, -28.91312939, -21.44305476, -30.78089926,
        -29.32587272, -40.59456656, -31.45460298, -41.12060034,
        -22.21136362, -49.23364735],
       [-26.28494431, -20.99019428, -17.55547709, -21.08924749,
        -22.43428031, -30.23122528, -24.39560619, -29.81375306,
        -15.67393281, -34.90511645],
       [-41.32566121, -30.46228738, -21.61966987, -34.34359518,
        -30.29260338, -43.44276798, -33.26718828, -44.23236602,
        -24.07951782, -53.41618964],
       [-36.95114087, -28.78742643, -22.56740776, -29.90795166,
        -29.95483944, -40.9123128 , -32.18918875, -40.90514467,
        -21.88527879, -48.51963654],
       [-52.31888988, -39.76144285, -30.47524021, -42.9072944 ,
        -40.88214041, -57.15106166, -44.78572498, -57.35103098,
        -30.76329762, -68.32955799],
       [-4

In [222]:
loss = tr.norm(B.inverse())**2
loss.backward()
B.grad.data


tensor([[  -11589.3276,   -29874.3660,   -19638.4680,   -68112.5576,
           -23138.0705,   -78361.6316,   -13223.4775,  -201379.6824,
           -28773.5468,   -86375.8003],
        [  -29870.0769,   -75395.9015,   -49590.3440,  -171139.0518,
           -58538.4345,  -196971.0847,   -33792.8672,  -504771.4357,
           -72505.9019,  -217245.1597],
        [  -19644.1754,   -49594.8915,   -32603.3747,  -112622.3582,
           -38492.0061,  -129595.7192,   -22200.1822,  -332169.1397,
           -47706.0370,  -142944.3031],
        [  -68112.1706,  -171142.2393,  -112617.0346,  -388062.7345,
          -132975.1423,  -446708.1334,   -76929.7411, -1144004.1919,
          -164523.0073,  -492731.4692],
        [  -23142.2304,   -58543.8527,   -38490.6717,  -132979.0182,
           -45433.5919,  -153023.8179,   -26189.9532,  -392302.4801,
           -56318.6303,  -168776.1027],
        [  -78367.7148,  -196979.4330,  -129593.2719,  -446713.4831,
          -153024.1067,  -514198.0555,   

In [225]:
np.linalg.inv(Bn).dot(np.linalg.inv(Bn).dot(np.linalg.inv(Bn)))*2

array([[  11564.92689608,   28938.36769688,   19060.35619763,
          65596.84699334,   22504.40100771,   75509.9161137 ,
          13035.40594266,  193281.84646491,   27831.52884104,
          83292.33499415],
       [  28938.36769688,   72429.28802749,   47698.66806261,
         164168.22822871,   56325.10057762,  188973.50298484,
          32626.44410813,  483729.46126709,   69658.52573867,
         208462.38634311],
       [  19060.35619763,   47698.66806261,   31416.61646826,
         108124.28323893,   37094.94759811,  124461.53591793,
          21486.35833434,  318587.97881687,   45876.99584712,
         137292.92420554],
       [  65596.84699334,  164168.22822871,  108124.28323893,
         372136.72585835,  127673.46028333,  428358.22583876,
          73949.87955332, 1096497.95885211,  157901.85702066,
         472532.73009952],
       [  22504.40100771,   56325.10057762,   37094.94759811,
         127673.46028333,   43803.4532857 ,  146962.3416337 ,
          25372.03768765